In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.baseline import BaselineNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [3]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear_1000.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/baseline_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/baseline_model


In [4]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear_1000.pkl


In [5]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [6]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [7]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [8]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, 5, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [9]:
# Load model and the optimizer
model = BaselineNonlinearModel(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 8.467e+08, Test loss: 1.108e+09, MSE(e): 5.296e+01, MSE(pi1): 2.566e+04, MSE(pi2): 1.880e+01, MSE(pi3): 6.049e+02
Epoch 100, Train loss: 4.130e+06, Test loss: 4.776e+06, MSE(e): 4.085e-01, MSE(pi1): 2.505e+00, MSE(pi2): 2.855e-01, MSE(pi3): 1.983e-01
Epoch 200, Train loss: 3.909e+06, Test loss: 4.347e+06, MSE(e): 3.888e-01, MSE(pi1): 7.409e-01, MSE(pi2): 2.792e-01, MSE(pi3): 1.328e-01
Epoch 300, Train loss: 3.958e+06, Test loss: 4.323e+06, MSE(e): 3.940e-01, MSE(pi1): 6.016e-01, MSE(pi2): 2.815e-01, MSE(pi3): 1.212e-01
Epoch 400, Train loss: 2.757e+06, Test loss: 2.836e+06, MSE(e): 2.737e-01, MSE(pi1): 9.157e-01, MSE(pi2): 1.847e-01, MSE(pi3): 1.096e-01
Epoch 500, Train loss: 2.609e+06, Test loss: 2.530e+06, MSE(e): 2.595e-01, MSE(pi1): 3.826e-01, MSE(pi2): 1.840e-01, MSE(pi3): 9.294e-02
Epoch 600, Train loss: 1.084e+05, Test loss: 1.672e+05, MSE(e): 9.799e-03, MSE(pi1): 4.122e-01, MSE(pi2): 6.419e-03, MSE(pi3): 6.230e-02
Epoch 700, 

In [10]:
# Parametros de entrenamiento
start_epoch = 9000
n_epochs = 100000

batch_size = 64
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 9000.
Epoch 9000, Train loss: 2.623e+03, Test loss: 2.016e+04, MSE(e): 2.231e-04, MSE(pi1): 1.143e-02, MSE(pi2): 1.298e-04, MSE(pi3): 2.770e-03
Epoch 9100, Train loss: 1.753e+03, Test loss: 1.870e+04, MSE(e): 1.426e-04, MSE(pi1): 7.449e-03, MSE(pi2): 7.777e-05, MSE(pi3): 2.529e-03
Epoch 9200, Train loss: 1.616e+03, Test loss: 1.819e+04, MSE(e): 1.301e-04, MSE(pi1): 7.452e-03, MSE(pi2): 7.318e-05, MSE(pi3): 2.406e-03
Epoch 9300, Train loss: 1.520e+03, Test loss: 1.784e+04, MSE(e): 1.216e-04, MSE(pi1): 7.388e-03, MSE(pi2): 6.988e-05, MSE(pi3): 2.306e-03
Epoch 9400, Train loss: 1.437e+03, Test loss: 1.757e+04, MSE(e): 1.143e-04, MSE(pi1): 7.260e-03, MSE(pi2): 6.693e-05, MSE(pi3): 2.214e-03
Epoch 9500, Train loss: 1.366e+03, Test loss: 1.738e+04, MSE(e): 1.082e-04, MSE(pi1): 7.124e-03, MSE(pi2): 6.435e-05, MSE(pi3): 2.129e-03
Epoch 9600, Train loss: 1.306e+03, Test loss: 1.725e+04, MSE(e): 1.031e-04, MSE(pi1): 7.007e-03, MSE(pi2): 6.209e-05, MSE(p